# The UVM Test

## The Singleton Design Pattern
Getting a single object instead of constructing new ones.

In [1]:
from pyuvm import *
class Vault(metaclass=Singleton):
    def __init__(self):
        self.key = 0x1D
        
class Crypto(uvm_object):
    def encode_decode(self, msg):
        assert (msg >= 0) and (msg < 256)
        vault = Vault()
        print(f"{self.get_name()} vault ID: {id(vault)}")
        return msg ^ vault.key

c1 = Crypto("encode")
c2 = Crypto("decode")


msg = c1.encode_decode(0x55)
print(f"{msg:02x}")
rcv = c2.encode_decode(msg)
print(rcv == 0x55)
print(f" Final vault ID: {id(Vault())}")
        

encode vault ID: 140196956134128
48
decode vault ID: 140196956134128
True
 Final vault ID: 140196956134128


In [2]:
help(uvm_object)

Help on class uvm_object in module pyuvm.s05_base_classes:

class uvm_object(pyuvm.utility_classes.uvm_void)
 |  uvm_object(name='')
 |  
 |  5.3.1
 |  
 |  Method resolution order:
 |      uvm_object
 |      pyuvm.utility_classes.uvm_void
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name='')
 |      Implements behavior in new()
 |      5.3.2
 |  
 |  clone(self)
 |      5.3.5.2
 |      Unlike the clone in the SystemVerilog UVM, this clone does the deep
 |      copy without the user needing to override do_copy()
 |  
 |  compare(self)
 |      5.3.9.1
 |  
 |  convert2string(self)
 |      5.3.6.4
 |      This interface exists in Python as the __str__() method
 |  
 |  copy(self)
 |      5.3.8.1
 |  
 |  do_compare(self)
 |      5.3.9.2
 |  
 |  do_copy(self)
 |      5.3.8.2
 |  
 |  do_execute_op(self, op)
 |      5.3.13.1
 |      :return:
 |  
 |  do_pack(self)
 |      5.3.10.2
 |      :return:
 |  
 |  do_print(self)
 |  
 |  do_record(self)
 |      5

## The Factory Design Pattern
Tiny game design example

In [3]:
import random
class Phaser:
    def sound_effect(self):
        print("Zzzap!")

class PhotonTorpedo:
    def sound_effect(self):
        print("Pew! Pew!")

factory_dict=dict(phaser=Phaser, torpedo=PhotonTorpedo)

for ff in range(6):
    weapon_no = random.randint(0,1)
    weapon_name = "phaser" if weapon_no == 0 else "torpedo"
    weapon = factory_dict[weapon_name]()
    weapon.sound_effect()

Zzzap!
Zzzap!
Zzzap!
Pew! Pew!
Pew! Pew!
Zzzap!


### The run\_phase Method

*Note: The code below needs to be run in a freshly started Jupyter notebook kernal. This means after you run it once, you must restart the kernal by pressing the circular arrow restart button above or selecting Kernal -> Restart in the menus to run it again.*

In [4]:
from pyuvm import *
from tb_pkg import *
import random
import time

class AluTest(uvm_test):
    @staticmethod
    def alu_op(A, B, op, is_dut = False):
        """TLM model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if is_dut:
            time.sleep(0.1)
            if is_dut and (random.randint(0,3) == 0):
                result = result + 1
        return result

    def run_phase(self):
        self.raise_objection()  ## You MUST raise an objection
        cvg = set() #functional coverage
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, is_dut=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        if len(set(Ops) - cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-cvg}")
        self.drop_objection()  ## drop the objection to end
uvm_root().run_test("AluTest")

INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: ae XOR 5f = 00f1
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: 1a MUL 2b = 045e
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: c5 ADD 20 = 00e5
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: df AND f7 = 00d7
ERROR: <ipython-input-4-639ec6314069>(38)[uvm_test_top]: FAILED: 92 ADD 94 = 0127 expected 0126
